<a href="https://colab.research.google.com/github/shirooo39/MiXLab/blob/master/MiXLabLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://github.com/shirooo39/MiXLab" target="_blank">
<img src="https://github.com/fluidicon.png" height="100px" align="left"></a>

# <font size=6><center>**Welcome to <font color=1DA261>Mi</font><font color=FFCf4C>XL</font><font color=4789F4>ab</font> Lite**</center></font>

<font size=4><center>This Notebook is a stripped down version of <a href="https://colab.research.google.com/github/shirooo39/MiXLab/blob/master/MiXLab.ipynb" target="_blank">MixLab</a>.</center></font>

# <font size=6><center>✦ *Colab Stay Alive* ✦</center></font>
---

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] Colab Stay Alive</font>
# @markdown This cell runs a JS code that will automatically press the reconnect button when you got disconnected due to idle.
# ================================================================ #

import IPython
from IPython.display import clear_output
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Clicked on the connect button"); 
     btn.click() 
     }
   
   btn = document.getElementById('connect')
   if (btn != null){
     console.log("Clicked on the reconnect button"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

clear_output()

# <font size=6><center>✦ *Mount/Unmount Google Drive* ✦</center></font>
---


In [ ]:
# ============================= FORM ============================= #
# @markdown <h3>← Mount/Unmount Google Drive</h3>
# @markdown <p>This cell will mount/unmount Google Drive to /content/drive/</p></br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
# ================================================================ #

from IPython.display import clear_output
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

clear_output()

In [ ]:
# @markdown <h3>← Force re-mount Google Drive</h3>

drive.mount("/content/drive", force_remount=True)

# <font size=6><center>✦ *aria2* ✦</center></font>
---


In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] aria2</font>
Aria2_rpc = True
Ariang_WEBUI = True
USE_FREE_TOKEN = True
TOKEN = ""
REGION = "US"
Tunneling = "argo_tunnel_(cloudflare)" #@param ["argo_tunnel_(cloudflare)", "ngrok"]

if Tunneling == "argo_tunnel_(cloudflare)":
    PORT_FORWARD = "argotunnel"
else:
    PORT_FORWARD = "ngrok"

# OUTPUT_DIR = "" # @param {type:"string"}
# ================================================================ #

import os, pathlib, zipfile, re
import urllib.request, requests
from IPython.display import HTML, clear_output
from urllib.parse import urlparse

PORT = 8221
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/mixlab.py"):
    hCode = "https://raw.githubusercontent.com/shirooo39/" \
            "MiXLab/master/resources/mixlab.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/mixlab.py")

from mixlab import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    CWD,
    displayUrl,
    findProcess,
    findPackageR
)

loadingAn()

# Setting up aria2
runSh('apt install -y aria2')
pathlib.Path('ariang').mkdir(mode=0o777, exist_ok=True)
pathlib.Path('downloads').mkdir(mode=0o777, exist_ok=True)

# Defining Github latest release tag
def latestTag(link):
    import re
    from urllib.request import urlopen
    htmlF = urlopen(link+"/releases/latest").read().decode('UTF-8')
    return re.findall(r'.+\/tag\/([.0-9A-Za-z]+)".+/', htmlF)[0]

# Downloading the latest version of ariaNg
if not os.path.exists("ariang/index.html"):
    # BASE_URL = r"https://github.com/mayswind/AriaNg"
    # LATEST_TAG = latestTag(BASE_URL)
    # urlF = f'{BASE_URL}/releases/download/{LATEST_TAG}/' \
    #        f'AriaNg-{LATEST_TAG}-AllInOne.zip'
    urllib.request.urlretrieve(findPackageR('mayswind/AriaNg', 'AllInOne.zip'), 'ariang/new.zip')
    with zipfile.ZipFile('ariang/new.zip', 'r') as zip_ref: zip_ref.extractall('ariang')
    try:
        pathlib.Path('ariang/new.zip').unlink()
    except FileNotFoundError:
        pass

# Starting up aria2 RPC and the WebUI (ariaNg)
try:
    if not OUTPUT_DIR:
        OUTPUT_DIR = f"downloads/"
    elif not os.path.exists(OUTPUT_DIR):
    
        clear_output()
    
        print("Unable to find the defined path!")
        exx()
except:
    OUTPUT_DIR = f"{CWD}/downloads/"

if Aria2_rpc:
    if not findProcess("aria2c", "--enable-rpc"):
        try:
            trackers = requests.get("https://trackerslist.com/best_aria2.txt").text
            cmdC = r"aria2c --enable-rpc --rpc-listen-port=6800 -D " \
                   fr"-d {OUTPUT_DIR} " \
                   r"-j 20 " \
                   r"-c " \
                   fr"--bt-tracker={trackers} " \
                   r"--bt-request-peer-speed-limit=0 " \
                   r"--bt-max-peers=0 " \
                   r"--seed-ratio=0.0 " \
                   r"--max-connection-per-server=10 " \
                   r"--min-split-size=10M " \
                   r"--follow-torrent=mem " \
                   r"--disable-ipv6=true " \
                   r" &"
            runSh(cmdC, shell=True)
        except:
            print("aria2 RPC is not enabled! Please enable the RPC first!")

# Configuring port forwarding
clear_output()

if Aria2_rpc:
    Server = PortForward_wrapper(PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['Aria2_rpc', 6800, 'http']], REGION.lower(), [f"{HOME}/.ngrok2/aria2.yml", 5042])
    data = Server.start('Aria2_rpc', displayB=False)
    Host = urlparse(data['url']).hostname
    port = "80"

clear_output()

if Ariang_WEBUI:
    if Aria2_rpc:
        filePath = 'ariang/index.html'
        with open(filePath, 'r+') as f:
            read_data = f.read()
            f.seek(0)
            f.truncate(0)
            read_data = re.sub(r'(rpcHost:"\w+.")|rpcHost:""', f'rpcHost:"{Host}"', read_data)
            read_data = re.sub(r'protocol:"\w+."', r'protocol:"ws"', read_data)
            read_data = re.sub(r'rpcPort:"\d+."', f'rpcPort:"{port}"', read_data)
            f.write(read_data)
    try:
        urllib.request.urlopen(f"http://localhost:{PORT}")
    except:
        runSh(f"python3 -m http.server {PORT} &", shell=True, cd="ariang/")
  
Server = PortForward_wrapper(PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['Ariang', PORT, 'http']], REGION.lower(), [f"{HOME}/.ngrok2/ariang.yml", 5043])
data2 = Server.start('Ariang', displayB=False)
data2['url'] = urlparse(data2['url'])._replace(scheme='http').geturl()
displayUrl(data2, pNamU='AriaNg : ')

if Aria2_rpc:
    display(HTML("""<style>@import url('https://fonts.googleapis.com/css?family=Source+Code+Pro:200,900');  :root {   --text-color: hsla(210, 50%, 85%, 1);   --shadow-color: hsla(210, 40%, 52%, .4);   --btn-color: hsl(210, 80%, 42%);   --bg-color: #141218; }  * {   box-sizing: border-box; } button { position:relative; padding: 10px 20px;     border: none;   background: none;      font-family: "Source Code Pro";   font-weight: 900;font-size: 100%;     color: var(--text-color);      background-color: var(--btn-color);   box-shadow: var(--shadow-color) 2px 2px 22px;   border-radius: 4px;    z-index: 0;overflow: hidden; -webkit-user-select: text;-moz-user-select: text;-ms-user-select: text;user-select: text;}  button:focus {   outline-color: transparent;   box-shadow: var(--btn-color) 2px 2px 22px; }  .right::after, button::after {   content: var(--content);   display: block;   position: absolute;   white-space: nowrap;   padding: 40px 40px;   pointer-events:none; }  button::after{   font-weight: 200;   top: -30px;   left: -20px; }   .right, .left {   position: absolute;   width: 100%;   height: 100%;   top: 0; } .right {   left: 66%; } .left {   right: 66%; } .right::after {   top: -30px;   left: calc(-66% - 20px);      background-color: var(--bg-color);   color:transparent;   transition: transform .4s ease-out;   transform: translate(0, -90%) rotate(0deg) }  button:hover .right::after {   transform: translate(0, -47%) rotate(0deg) }  button .right:hover::after {   transform: translate(0, -50%) rotate(-7deg) }  button .left:hover ~ .right::after {   transform: translate(0, -50%) rotate(7deg) }  /* bubbles */ button::before {   content: '';   pointer-events: none;   opacity: .6;   background:     radial-gradient(circle at 20% 35%,  transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 75% 44%, transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 46% 52%, transparent 0, transparent 4px, var(--text-color) 5px, var(--text-color) 6px, transparent 6px);    width: 100%;   height: 300%;   top: 0;   left: 0;   position: absolute;   animation: bubbles 5s linear infinite both; }  @keyframes bubbles {   from {     transform: translate();   }   to {     transform: translate(0, -66.666%);   } }.zui-table {    border: solid 1px #DDEEEE;    border-collapse: collapse;    border-spacing: 0;    font: normal 13px;}.zui-table thead th {    background-color: #DDEFEF;    border: solid 1px #DDEEEE;    color: #0000009e;    padding: 10px;    text-align: left;}.zui-table tbody td {border: solid 1px #effff97a;color: #ffffffd1;    padding: 10px;}</style><center><button><table class="zui-table blueBG"><p>aria2 RPC Configuration<p><thead>        <tr>            <th>Protocol</th>            <th>Host</th>            <th>Port</th>        </tr>    </thead>    <tbody>        <tr>            <td>WebSocket</td><td>"""+Host+"""</td><td>"""+port+"""</td></tr></tbody></table><p>Click <a target="_blank" style="text-decoration: none;color: hsla(210, 50%, 85%, 1);font-size: 10px;" href="http://bit.ly/2qoOw0a">HERE</a> to see how to use the configuration.</p></button><center>"""))